# PyTorch Introduction: Cats and Dogs Classification
In this notebook, we'll build a simple CNN classifier to distinguish between cats and dogs. We'll cover:
1. Loading and preprocessing data using custom Dataset class
2. Building a CNN model
3. Training and validation loops
4. Visualizing results



# 1. Import Required Libraries

In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
import kagglehub
import os
from PIL import Image
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np


# 2. Create Custom Dataset Class
 
The Dataset class is responsible for:
- Loading individual images
- Assigning labels (0 for cats, 1 for dogs)
- Applying transformations
- Providing length information



In [ ]:
# Download the dataset
path = kagglehub.dataset_download("marquis03/cats-and-dogs")

In [ ]:
print(os.listdir(path))

In [ ]:
class CatsDogsDataset(Dataset):
    def __init__(self, root_dir, split='train', transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.split = split
        

        csv_file = os.path.join(root_dir, f'{split}.csv')
        self.data = pd.read_csv(csv_file)
        
        # Extract image paths and labels
        self.image_paths = self.data['image:FILE'].values
        self.labels = self.data['category'].values
    
    def __len__(self):
        return len(self.image_paths)
    
    def __getitem__(self, idx):
        # Construct full image path
        img_path = os.path.join(self.root_dir, self.image_paths[idx])
        
        # Read image
        image = Image.open(img_path).convert('RGB')
        label = self.labels[idx]
        
        # Apply transforms if any
        if self.transform:
            image = self.transform(image)
            
        return image, label

# 3. Define CNN Model Architecture
 
Our CNN model consists of:
- 3 convolutional layers with ReLU activation and max pooling
- Flatten layer to convert 2D features to 1D
- 2 fully connected layers
- Dropout for regularization
- Sigmoid activation for binary classification


In [ ]:
class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        self.conv_layers = nn.Sequential(
            nn.Conv2d(3, 16, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),
            
            nn.Conv2d(16, 32, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),
            
            nn.Conv2d(32, 64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2, 2)
        )
        
        self.fc_layers = nn.Sequential(
            nn.Flatten(),
            nn.Linear(64 * 28 * 28, 512),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(512, 1),
            nn.Sigmoid()
        )
        
    def forward(self, x):
        x = self.conv_layers(x)
        x = self.fc_layers(x)
        return x



# 4. Data Preprocessing and Loading
Here we:
- Define image transformations
- Create train and validation datasets
- Create DataLoader instances for batch processing

In [ ]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], # These random ahh values were found using ImageNet dataset, https://paperswithcode.com/dataset/imagenet
                          std=[0.229, 0.224, 0.225])
])

# Create train and validation datasets
train_dataset = CatsDogsDataset(path, split='train', transform=transform)
val_dataset = CatsDogsDataset(path, split='val', transform=transform)

# Create dataloaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=True)

# 4.2 Checking our images
Here we will load the first image of our data set and plot it along with its class

In [ ]:
img, label = train_dataset[270]

# Convert tensor to numpy for visualization
img_np = img.numpy().transpose(1, 2, 0)

# Denormalize the image
mean = np.array([0.485, 0.456, 0.406])
std = np.array([0.229, 0.224, 0.225])
img_np = std * img_np + mean
img_np = np.clip(img_np, 0, 1)

# Plot the image
plt.figure(figsize=(8, 8))
plt.imshow(img_np)
plt.title(f'Class: {"Cat" if label == 0 else "Dog"}')
plt.axis('off')
plt.show()


# 5. Model Setup
Initialize the model, loss function, and optimizer


In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = SimpleCNN().to(device)
criterion = nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)


# 6. Training and Validation Functions

Define functions for:
- Training one epoch
- Validating the model

Both functions return loss and accuracy metrics


In [ ]:
def train_epoch(model, train_loader, criterion, optimizer, device):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0
    
    for images, labels in train_loader:
        images, labels = images.to(device), labels.float().to(device)
        
        # Zero the gradients
        optimizer.zero_grad()
        
        # Forward pass
        outputs = model(images).squeeze()
        loss = criterion(outputs, labels)
        
        # Backward pass and optimize
        loss.backward()
        optimizer.step()
        
        # Statistics
        running_loss += loss.item()
        predicted = (outputs > 0.5).float()
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    
    epoch_loss = running_loss / len(train_loader)
    accuracy = 100 * correct / total
    return epoch_loss, accuracy

def validate(model, val_loader, criterion, device):
    model.eval()
    running_loss = 0.0
    correct = 0
    total = 0
    
    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.float().to(device)
            
            outputs = model(images).squeeze()
            loss = criterion(outputs, labels)
            
            running_loss += loss.item()
            predicted = (outputs > 0.5).float()
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    
    epoch_loss = running_loss / len(val_loader)
    accuracy = 100 * correct / total
    return epoch_loss, accuracy


# 7. Training Loop
 
Train the model for specified number of epochs and track metrics



In [ ]:
# Training
num_epochs = 20
train_losses = []
val_losses = []
train_accuracies = []
val_accuracies = []

for epoch in range(num_epochs):
    # Train
    train_loss, train_acc = train_epoch(model, train_loader, criterion, optimizer, device)
    train_losses.append(train_loss)
    train_accuracies.append(train_acc)
    
    # Validate
    val_loss, val_acc = validate(model, val_loader, criterion, device)
    val_losses.append(val_loss)
    val_accuracies.append(val_acc)
    
    print(f'Epoch [{epoch+1}/{num_epochs}]')
    print(f'Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.2f}%')
    print(f'Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.2f}%')
    print('-' * 50)


# 8. Visualize Results
 
Plot training and validation metrics over time

In [ ]:

plt.figure(figsize=(12, 4))

plt.subplot(1, 2, 1)
plt.plot(train_losses, label='Train Loss')
plt.plot(val_losses, label='Val Loss')
plt.title('Loss vs Epoch')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(train_accuracies, label='Train Accuracy')
plt.plot(val_accuracies, label='Val Accuracy')
plt.title('Accuracy vs Epoch')
plt.xlabel('Epoch')
plt.ylabel('Accuracy (%)')
plt.legend()

plt.tight_layout()
plt.show()

# Very nice
Now you can detect all cats and dogs(As long as they are in the training set)

![No cat is safe](https://i.imgur.com/1r243HR.gif)